<a href="https://colab.research.google.com/github/TomasRojass/TP-Progra-concurrente/blob/main/TP1/Parte%203/Tp1_parte_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP 1 Parte 3

## java


In [ ]:
%%writefile Process.java
import java.util.*;
import java.util.concurrent.ArrayBlockingQueue;

public class ProducerConsumerExample
{
    public static void main(String[] args)
    {
      if (args.length != 1)
      {
          System.out.println("Ejemplo de uso: java ProducerConsumerExample <numOfRandomNumbers>");
          return;
      }

        int numOfRandomNumbers = Integer.parseInt(args[0]);
        ArrayBlockingQueue<Integer> queue = new ArrayBlockingQueue<>(numOfRandomNumbers); // Create a blocking queue with a capacity of 10

        Thread producerThread = createProducerThread(queue,numOfRandomNumbers);
        Thread consumerThread = createConsumerThread(queue,numOfRandomNumbers);

        // Start both threads
        producerThread.start();
        consumerThread.start();
    }

    public static int generateRandomNumber()
    {
        return (int) (Math.random() * 100);
    }

    public static Thread createProducerThread(ArrayBlockingQueue<Integer> queue, int numOfRandomNumbers)
    {
        return new Thread(() ->
        {
            try
            {
                for (int i = 0; i < numOfRandomNumbers; i++)
                {
                    int randomNum = generateRandomNumber();
                    System.out.println("Producing: " + randomNum);
                    queue.put(randomNum);
                }
            }
            catch (InterruptedException e)
            {
                Thread.currentThread().interrupt();
            }
        });
    }

    public static Thread createConsumerThread(ArrayBlockingQueue<Integer> queue, int numOfRandomNumbers)
    {
        return new Thread(() ->
        {
            Map<Integer, Integer> consumedNumbers = new HashMap<>(); // Map to store frequencies
            int sum = 0;
            try
            {
                for (int i = 0; i < numOfRandomNumbers; i++)
                {
                    int num = queue.take();
                    sum += num;
                    consumedNumbers.put(num, consumedNumbers.getOrDefault(num, 0) + 1);
                }
                double average = (double) sum / numOfRandomNumbers;
                int maximum = Collections.max(consumedNumbers.keySet());
                int minimum = Collections.min(consumedNumbers.keySet());
                List<Integer> mostFrequentValues = findMostFrequentValues(consumedNumbers);

                System.out.println("Promedio: " + average);
                System.out.println("Maximo: " + maximum);
                System.out.println("Minimo: " + minimum);
                System.out.println("Suma: " + sum);
                System.out.println("El mas frecuente: " + mostFrequentValues);

            }
            catch (InterruptedException e)
            {
                Thread.currentThread().interrupt();
            }
        });
    }

    public static List<Integer> findMostFrequentValues(Map<Integer, Integer> frequencyMap)
    {
        List<Integer> mostFrequentValues = new ArrayList<>();
        int maxFrequency = 0;

        for (Map.Entry<Integer, Integer> entry : frequencyMap.entrySet())
        {
            int frequency = entry.getValue();

            if (frequency > maxFrequency)
            {
                mostFrequentValues.clear();
                mostFrequentValues.add(entry.getKey());
                maxFrequency = frequency;
            }
            else if (frequency == maxFrequency)
            {
                mostFrequentValues.add(entry.getKey());
            }
        }

        return mostFrequentValues;
    }
}


Overwriting Process.java


In [ ]:
!java Process.java 6

Producing: 1
Producing: 1
Producing: 1
Producing: 1
Producing: 0
Producing: 1
Promedio: 0.8333333333333334
Maximo: 1
Minimo: 0
Suma: 5
El mas frecuente: [1]


## Python

In [ ]:
import multiprocessing
import os
import sys
import pprint


def calculate_stats(wordlist):
    res = {
        "Total Caracteres": 0,
        "Total Letras": 0,
        "Total Digitos": 0,
        "Palabra mas Corta": None,
        "Palabra mas Larga": None,
    }

    for i in wordlist:
        res["Total Caracteres"] += len(i)
        res["Total Letras"] += amount_letters(i)
        res["Total Digitos"] += amount_digits(i)

    wordlist_by_len = sorted(wordlist, key=len)
    res["Palabra mas Corta"] = wordlist_by_len[0]
    res["Palabra mas Larga"] = wordlist_by_len[-1]

    return res


def amount_digits(word):
    res = 0
    for i in word:
        if i.isdigit():
            res += 1
    return res


def amount_letters(word):
    res = 0
    for i in word:
        if i.isalpha():
            res += 1
    return res


def main():
    CHILD = 0

    child_pipe, parent_pipe = multiprocessing.Pipe(duplex=True)

    process = os.fork()
    if process < 0:
        sys.exit("Error al crear el nuevo proceso")

    if process == CHILD:
        child_done = False
        words = []
        while not child_done:
            rec = child_pipe.recv()
            if rec.lower() == "close":
                child_done = True
            else:
                words.append(rec)
        child_pipe.send(calculate_stats(words))

        parent_pipe.close()
        child_pipe.close()
        os._exit(0)

    else:
        parent_done = False
        while not parent_done:
            inp = input(">>>")
            parent_pipe.send(inp)
            if inp.lower() == "close":
                parent_done = True
        stats = parent_pipe.recv()
        pprint.pp(stats)
        parent_pipe.close()
        child_pipe.close()
        os.wait()

if __name__ == "__main__":
    main()


>>>perro
>>>gato
>>>mono12
>>>ooo0000''''''
>>>close
{'Total Caracteres': 28,
 'Total Letras': 16,
 'Total Digitos': 6,
 'Palabra mas Corta': 'gato',
 'Palabra mas Larga': "ooo0000''''''"}


## C++

In [6]:
%%writefile password.cpp
#include <iostream>
#include <string.h>
#include <thread>
#include <mutex>

std::mutex mtx;
int pos = 0;
char *args = NULL;
int result[50] = {};

int getNumberFromChar(char letter){
  if(letter >= 'a' && letter <= 'z'){
    return int(letter)-96;
  }
  if(letter >= 'A' && letter <= 'Z'){
    return int(letter)-64;
  }
  return -1;
}

void getPassword(int cant) {
  std::cout << "TID: " << std::this_thread::get_id() << std::endl;
  std::cout << "Cantidad de letras a procesar: " << cant << std::endl << std::endl;

  for(int i=0; i<cant; i++){
    mtx.lock();
    int number = getNumberFromChar(args[pos]);
    result[pos] = number;
    pos++;
    mtx.unlock();
  }
}

int main(int argc, char **argv) {

  args = argv[1];
  int cant1 = strlen(argv[1]) / 2;
  int cant2 = strlen(argv[1]) - cant1;

  std::thread thread_1(getPassword, cant1);
  std::thread thread_2(getPassword, cant2);

  thread_1.join();
  thread_2.join();

  std::cout << "password numbers: " << std::endl;
  for(int i=0; i<strlen(argv[1]); i++)
  {
    std::cout << result[i] << std::endl;
  }

  return EXIT_SUCCESS;
}


Overwriting password.cpp


In [7]:
!gcc -o password ./password.cpp -lstdc++

In [8]:
!./password ARBOLITOS

TID: 136346498926144
Cantidad de letras a procesar: 5

TID: 136346507318848
Cantidad de letras a procesar: 4

password numbers: 
1
18
2
15
12
9
20
15
19
